<a href="https://colab.research.google.com/github/coding-ss/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/DS_Unit_1_Sprint_Challenge_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html]) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [0]:
import pandas as pd

cell_phones = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv')

population = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv')

geo_country_codes = (pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')
                       .rename(columns={'country': 'geo', 'name': 'country'}))

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [2]:
cell_phones.head()

,geo,time,cell_phones_total
0,abw,1960,0.0
1,abw,1965,0.0
2,abw,1970,0.0
3,abw,1975,0.0
4,abw,1976,0.0


In [3]:
population.head()

,geo,time,population_total
0,afg,1800,3280000
1,afg,1801,3280000
2,afg,1802,3280000
3,afg,1803,3280000
4,afg,1804,3280000


In [0]:
merged = pd.merge(population,cell_phones,how='inner', on=('geo','time'))

In [7]:
merged.shape

(8590, 4)

Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

In [8]:
geo_country_codes.head()

,geo,alt_5,alternative_1,alternative_2,alternative_3,alternative_4_cdiac,arb1,arb2,arb3,arb4,...,latitude,longitude,main_religion_2008,country,pandg,un_state,unicode_region_subtag,upper_case_name,world_4region,world_6region
0,abkh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Abkhazia,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,NaN,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,...,12.50000,-69.96667,christian,Aruba,NaN,False,AW,ARUBA,americas,america
2,afg,NaN,Islamic Republic of Afghanistan,NaN,NaN,Afghanistan,NaN,NaN,NaN,NaN,...,33.00000,66.00000,muslim,Afghanistan,AFGHANISTAN,True,AF,AFGHANISTAN,asia,south_asia
3,ago,NaN,NaN,NaN,NaN,Angola,NaN,NaN,NaN,NaN,...,-12.50000,18.50000,christian,Angola,ANGOLA,True,AO,ANGOLA,africa,sub_saharan_africa
4,aia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.21667,-63.05000,christian,Anguilla,NaN,False,AI,ANGUILLA,americas,america


In [0]:
final = pd.merge(merged,geo_country_codes[['geo','country']])

In [13]:
final.shape

(8590, 5)

In [18]:
final

,geo,time,population_total,cell_phones_total,country
0,afg,1960,8996351,0.0,Afghanistan
1,afg,1965,9938414,0.0,Afghanistan
2,afg,1970,11126123,0.0,Afghanistan
3,afg,1975,12590286,0.0,Afghanistan
4,afg,1976,12840299,0.0,Afghanistan
5,afg,1977,13067538,0.0,Afghanistan
6,afg,1978,13237734,0.0,Afghanistan
7,afg,1979,13306695,0.0,Afghanistan
8,afg,1980,13248370,0.0,Afghanistan
9,afg,1981,13053954,0.0,Afghanistan


In [31]:
final['country'].value_counts().head(500)

Togo                    46
Panama                  46
Rwanda                  46
Hong Kong, China        46
Sri Lanka               46
Malawi                  46
Azerbaijan              46
Uganda                  46
Georgia                 46
Philippines             46
Finland                 46
Fiji                    46
Bulgaria                46
Slovak Republic         46
Niger                   46
Trinidad and Tobago     46
Germany                 46
Honduras                46
Cyprus                  46
Cape Verde              46
Zimbabwe                46
Hungary                 46
Switzerland             46
Seychelles              46
Guyana                  46
Syria                   46
China                   46
Angola                  46
France                  46
Mozambique              46
                        ..
North Korea             45
Dominica                45
South Korea             44
Tuvalu                  44
Guinea                  44
Tunisia                 44
S

## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [17]:
cell_phones_per_person = len('cell_phones_total')/len('population_total')
cell_phones_per_person

1.0625

In [185]:
final['cell_phones_per_person'] = cell_phones_per_person
final.head()

,geo,time,population_total,cell_phones_total,country,cell_phones_per_person
0,AFG,1960,8996351,0.0,Afghanistan,1.0625
1,AFG,1965,9938414,0.0,Afghanistan,1.0625
2,AFG,1970,11126123,0.0,Afghanistan,1.0625
3,AFG,1975,12590286,0.0,Afghanistan,1.0625
4,AFG,1976,12840299,0.0,Afghanistan,1.0625


In [46]:
final.loc[(final['country'] == 'United States') & (final['time'] == 2017)]


,geo,time,population_total,cell_phones_total,country
8134,USA,2017,324459463,395881000.0,United States


In [47]:
395881000.0/324459463

1.2201246847283354

Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [182]:
final['geo'] = final['geo'].str.upper()
final['geo'].head()

final.head()

,geo,time,population_total,cell_phones_total,country
0,AFG,1960,8996351,0.0,Afghanistan
1,AFG,1965,9938414,0.0,Afghanistan
2,AFG,1970,11126123,0.0,Afghanistan
3,AFG,1975,12590286,0.0,Afghanistan
4,AFG,1976,12840299,0.0,Afghanistan


## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [32]:
final.describe()

,time,population_total,cell_phones_total
count,8590.000000,8.590000e+03,8.590000e+03
mean,1994.193481,2.983823e+07,9.004950e+06
std,14.257975,1.161284e+08,5.573408e+07
min,1960.000000,4.433000e+03,0.000000e+00
25%,1983.000000,1.456148e+06,0.000000e+00
50%,1995.000000,5.725062e+06,6.200000e+03
75%,2006.000000,1.810581e+07,1.697652e+06
max,2017.000000,1.409517e+09,1.474097e+09


In [36]:
# To show all numeric columns

import numpy as np 

final.describe(include=[np.number])

,time,population_total,cell_phones_total
count,8590.000000,8.590000e+03,8.590000e+03
mean,1994.193481,2.983823e+07,9.004950e+06
std,14.257975,1.161284e+08,5.573408e+07
min,1960.000000,4.433000e+03,0.000000e+00
25%,1983.000000,1.456148e+06,0.000000e+00
50%,1995.000000,5.725062e+06,6.200000e+03
75%,2006.000000,1.810581e+07,1.697652e+06
max,2017.000000,1.409517e+09,1.474097e+09


In [37]:
# To show all non-numeric columns

final.describe(include=[np.object])

,geo,country
count,8590,8590
unique,195,195
top,ZMB,Togo
freq,46,46


In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [0]:
f = final['time'] == 2017

In [281]:
f2 = final[f]
f2.head()

,geo,time,population_total,cell_phones_total,country,cell_phones_per_person
45,AFG,2017,35530081,"23,929,713.0",Afghanistan,1.0625
91,AGO,2017,29784193,"13,323,952.0",Angola,1.0625
137,ALB,2017,2930187,"3,497,950.0",Albania,1.0625
183,AND,2017,76965,"80,337.0",Andorra,1.0625
219,ARE,2017,9400145,"19,826,224.0",United Arab Emirates,1.0625


In [263]:
f3 = f2.sort_values(by='cell_phones_total', ascending=False).head()
f3

,geo,time,population_total,cell_phones_total,country,cell_phones_per_person
1496,CHN,2017,1409517397,"1,474,097,000.0",China,1.0625
3595,IND,2017,1339180127,"1,168,902,277.0",India,1.0625
3549,IDN,2017,263991379,"458,923,202.0",Indonesia,1.0625
8134,USA,2017,324459463,"395,881,000.0",United States,1.0625
1084,BRA,2017,209288278,"236,488,548.0",Brazil,1.0625


In [315]:
# Answer

f4 = f3[['cell_phones_total','country']]

f4.columns = f4.columns.str.replace('_',' ')
f4

,cell phones total,country
1496,"1,474,097,000.0",China
3595,"1,168,902,277.0",India
3549,"458,923,202.0",Indonesia
8134,"395,881,000.0",United States
1084,"236,488,548.0",Brazil


In [0]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

In [0]:

f = final.query('cell_phones_total > population_total')

In [0]:
# In 2014, US had more cell phones than number of people

f.tail(50)

## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

In [0]:

time1_df = final.loc[(final['time'] >= 2007) & (final['time'] <= 2017)]
time1

In [0]:
subset = time1[time1['country'].isin(['China', 'India', 'United States', 'Brazil','Indonesia']) ]

In [306]:
table1 = subset.pivot_table(values = 'cell_phones_total',
                          index=['country'],
                          columns = ['time'])

table1.head()

time,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
country,,,,,,,,,,,
Brazil,"120,980,103.0","150,641,403.0","169,385,584.0","196,929,978.0","234,357,507.0","248,323,703.0","271,099,799.0","280,728,796.0","257,814,274.0","244,067,356.0","236,488,548.0"
China,"547,306,000.0","641,245,000.0","747,214,000.0","859,003,000.0","986,253,000.0","1,112,155,000.0","1,229,113,000.0","1,286,093,000.0","1,291,984,200.0","1,364,934,000.0","1,474,097,000.0"
India,"233,620,000.0","346,890,000.0","525,090,000.0","752,190,000.0","893,862,478.0","864,720,917.0","886,304,245.0","944,008,677.0","1,001,056,000.0","1,127,809,000.0","1,168,902,277.0"
Indonesia,"93,386,881.0","140,578,243.0","163,676,961.0","211,290,235.0","249,805,619.0","281,963,665.0","313,226,914.0","325,582,819.0","338,948,340.0","385,573,398.0","458,923,202.0"
United States,"249,300,000.0","261,300,000.0","274,283,000.0","285,118,000.0","297,404,000.0","304,838,000.0","310,698,000.0","355,500,000.0","382,307,000.0","395,881,000.0","395,881,000.0"


#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

If you have the time and curiosity, what other questions can you ask and answer with this data?